# Heterogeneous mixing introduction
Heterogeneous mixing between population sub-groups
can be one of the most difficult areas of
compartmental modelling of infectious disease transmission.
For this reason, we'll work through this over the course of several notebooks.
Let's start off with what heterogeneous mixing is,
why we we might want to include it in our models and 
what we'll be assuming and the notation we'll be using.

## Rationale
Direct transmission of infectious pathogens is driven by interactions between 
individuals within the human population we're simulating.
For the previous models we've been simulating,
we have been assuming homogeneous mixing.
This assumption means that any two individuals within the population
have the same chance of coming into contact and transmitting infection.
This is analogous to how gas particles interact in Brownian motion,
but may not adequately represent how humans interact in society.

In reality, there is heterogeneity in the rate of contact between people
in our population of interest, which may be driven by a range of factors.
One way of simulating this would be to represent every person in the population separately, 
which is an increasingly commonly adopted approach, called agent-based or network modelling.
For now, we're going to keep focusing on compartmental models,
and so we'll have to represent our population sub-groups as compartments.

Even with just a compartmental model,
it is still possible to incorporate heterogeneous mixing 
between our population groups.
If we implement heterogeneous mixing through model stratification,
we're still limited to assuming that the rates of infection for each stratum
are the same and can have one rate of interaction 
between each combination of two strata.
However, we can stratify our population into as many sub-groups as we like,
including applying multiple model stratifications to represent multiple 
population characteristics.
At some point, it might actually become more efficient 
to use an agent-based approach,
perhaps if the number of compartments 
exceeds the number of people in the population,
but we'll start off with the simplest 
heterogeneous mixing stratifications possible
to illustrate the general points.

## Assumptions and conventions
Throughout these sections on heterogeneous mixing, let's maintain the key assumptions
that our infection is directly-transmitted and our stratifications are complete and mutually exclusive.
That is, any population heterogeneity we're interested in will apply to the entire population,
and our stratifications should therefore be applied to the entire population
(so if we implement a stratification to capture a "rural" group within our population,
we should also implement an "urban" stratum to represent the rest of the population).
Also, for consistency, we'll generally order our groups sequentially from 
left to right and from top to bottom.
For example, this would imply that cell with index $(0, 0)$ 
contains information on the rates of contact between people from 
the first modelled sub-population and others from the same group.

Let's have a really quick look at the syntax for 
how we would implement a stratification in `summer`
(but at this stage without implementing any heterogeneous mixing
or building a meaningful model).

In [ ]:
# If running on Google Colab, run the following line of code to install the summer package
# %pip install summerepi2

In [ ]:
from jax import numpy as jnp

from summer2 import CompartmentalModel, Stratification
from summer2.parameters import Parameter

In [ ]:
compartments = (
    "susceptible", 
    "infectious", 
)
model = CompartmentalModel(
    times=(0.0, 40.0),
    compartments=compartments,
    infectious_compartments=("infectious",),
)    
strat = Stratification(
    "groups",
    ["group1", "group2"],
    compartments,
)
model.stratify_with(strat)

## Heterogeneous mixing matrix
Let's now introduce what we mean by a mixing matrix,
and be very clear on what we are representing before we get started
thinking about heterogeneous mixing.
In the previous example, we had two population subgroups,
although we could have any number, of course.
Each of our two population subgroups or strata 
can interact with either of the other two population subgroups.
We'll have to define values for the rate of interaction
between each pair of population groups,
including the interaction between a stratum and itself.
Therefore, the number of types of interactions between 
the members of population strata/groups is $n^{2}$, 
where $n$ is the number of strata we are simulating.
In the [next notebook](./13-mixing-and-transmission-types.ipynb), we'll come back to what we mean by these values in some detail,
but for the moment, let's just think of these as providing some information 
on the strength or rate of interaction between the subgroups.
`summer` needs the user to supply a matrix or array to represent these values,
which should be a square matrix with both the rows and columns ordered
according to the strata being implemented in the stratification process.

Our convention (which is common in infectious disease modelling)
will be that the rows represent the population being infected,
while the columns represent the population doing the infecting.
If we refer to an element of the matrix as $\beta_{i, j}$,
then $i$ represents the group that is susceptible to infection and being infected,
while $j$ represents the group containing infectious people and doing the infecting.
This might seem counter-intuitive, 
because we can also think of this as the infection process proceeding backwards,
or going from $j$ to $i$.
However, if we are going to make the infectors the columns and the infectees the rows,
we definitely want to go this way round because we should always notate
elements of a matrix with the row subscript before the column subscript.

Last, let's just show the syntax for doing this in `summer`,
with a trivial heterogeneous mixing stratification
applied to an unrealistic model.

In [ ]:
def build_hetero_mix_model():
    compartments = (
        "susceptible", 
        "infectious",
    )
    model = CompartmentalModel(
        times=(0.0, 40.0),
        compartments=compartments,
        infectious_compartments=("infectious",),
    )
    model.set_initial_population({"susceptible": 1.0})
    model.add_infection_density_flow(
        "infection",
        Parameter("risk_per_contact"),
        source="susceptible", 
        dest="infectious",
    )
    strat = Stratification(
        "groups",
        ["group1", "group2"],
        compartments,
    )
    mixing_matrix = jnp.array(
        [
            [1.0, 1.0],  # Values for group1 and group2 infecting group 1, respectively
            [1.0, 1.0],  # Values for group1 and group2 infecting group 2, respectively
        ]
    )
    strat.set_mixing_matrix(mixing_matrix)
    model.stratify_with(strat)
    return model

In [ ]:
mix_model = build_hetero_mix_model()
mix_model.run({"risk_per_contact": 1.0})

## Equivalent equations
For those more familiar with notating these processes using equations,
let's write out the equation for the force of infection in `group1`:
$$ \lambda _{1} = \beta_{1, 1} I_{1}(t) + \beta_{1, 2} I_{2}(t) $$
From this, we can see that the force of infection for `group1`
is comprised of two components - transmission from other members of `group1`
and transmission from members of `group2`.
Of course, we can make things more complicated later,
and we would probably want to in order to explore any interesting dynamics,
but we won't keep coming back to the equations for these processes,
because `summer` implements all of this behind the scenes and 
we prefer to let the code speak for itself.
It should be easy to write down the equivalent equation for
the force of infection for `group2`.